In [2]:
using WGMODES
using Plots

# パラメータの設定
lambda = 1.55 # 波長
dx = dy = 0.02
x = -2:dx:2
y = -1:dy:1

# 誘電率の分布を定義
index = [y_val < 0 ? 1.5 : (y_val < 0.2 || (y_val < 0.5 && abs(x_val) < 1) ? 3.45 : 1.0) for y_val in y, x_val in x]
epsilon = index .^ 2
epsilon_r = epsilon

# モード計算のパラメータ
guess = maximum(index)
nmodes = 1
boundary = "0000"

# モード計算
hx, hy, neff = WGMODES.wgmodes(lambda, guess, nmodes, dx, dy, epsilon, boundary);

# 定数の定義
c0 = 299792458            # 真空中の光速 (m/s)
mu0 = 4π * 1e-7           # 真空の透磁率 (H/m)
epsilon0 = 1 / (mu0 * c0^2)  # 真空の誘電率 (F/m)
omega = 2π * c0 / lambda    # 角周波数 (rad/s)

# 伝搬定数の計算
k0 = 2π / lambda            # 真空中の波数
beta = neff[1] * k0         # 最初のモードの伝搬定数

# 配列サイズの確認
println("Size of hx[:, :, 1]: ", size(hx[:, :, 1]))
println("Size of hy[:, :, 1]: ", size(hy[:, :, 1]))
println("Size of epsilon_r: ", size(epsilon_r))

@assert size(hx[:, :, 1]) == size(epsilon_r) "Size mismatch between hx and epsilon_r"
@assert size(hy[:, :, 1]) == size(epsilon_r) "Size mismatch between hy and epsilon_r"

# 電界成分の計算（ベクトル化）
epsilon_point = epsilon_r * epsilon0
Ex = (beta ./ (omega .* epsilon_point)) .* hy[:, :, 1]
Ey = (beta ./ (omega .* epsilon_point)) .* hx[:, :, 1]

# プロット
Plots.plot(
    Plots.heatmap(x, y, index',
        title="Waveguide Index Structure",
        aspectratio=1,
    ),
    Plots.heatmap(x, y, real.(Ex)',
        title="Electric Field Distribution (E_x)",
        aspectratio=1,
    ),
    Plots.heatmap(x, y, real.(Ey)',
        title="Electric Field Distribution (E_y)",
        aspectratio=1,
    ),
    size=(800, 600),
)


Size of hx[:, :, 1]: (102, 202)
Size of hy[:, :, 1]: (102, 202)
Size of epsilon_r: (101, 201)


AssertionError: AssertionError: Size mismatch between hx and epsilon_r